In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import Matrix, Array
from sympy.abc import x, y, z
import torch
from torch.autograd.functional import jacobian
import matplotlib.pyplot as plt

from matplotlib.collections import LineCollection
import matplotlib.colors as colors
import cProfile
import pstats
import warnings

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
torch.set_default_dtype(torch.float64)

In [ ]:
# Implementing SHAKE and RATTLE

In [ ]:
@torch.compile(mode = "default")
def G(gs):
    '''
    :param gs: a list of tensor functions
    :return: a function sending a tensor to the stacked matrix of the functions of that tensor
    '''

    def G_gs(tensor):
        # print("Function input: ",tensor) # checking the input for debugging
        # print("Function output:" , torch.stack([g(tensor) for g in gs],0))
        return torch.stack([g(tensor) for g in gs], 0)

    return G_gs

In [ ]:
@torch.compile(mode = "default")
def J(gs, x):
    '''Returns the Jacobian evaluated at x for a list gs of constraint functions'''
    return jacobian(G(gs), x)

In [ ]:
@torch.compile(mode = "default")
def rattle_step(x, v1, h, M, gs, e):
    '''
    Defining a function to take a step in the position, velocity form.
    g should be a vector-valued function of constraints.
    :return: x_1, v_1
    '''

    M1 =  torch.inverse(M)

    G1 = G(gs)


    DV = torch.zeros_like(x)

    #DV[-1] = 10  # leaving this out for g-BAOAB
    DV_col = DV.reshape(-1, 1)

    x_col = x.reshape(-1, 1)
    v1_col = v1.reshape(-1, 1)

    # doing Newton-Raphson iterations
    iters = 0
    x2 = x_col + h * v1_col - 0.5*(h**2)* M1 @ DV_col
    Q_col = x2
    Q = torch.squeeze(Q_col)
    J1 = J(gs, torch.squeeze(x_col))


    #while torch.any(torch.abs(G1(Q)) > e):
    for i in range(3):
        J2 = J(gs, torch.squeeze(Q))
        R = J2 @ M1 @ J1.t()
        dL = torch.inverse(R) @ G1(Q)
        #print(f"Q = {Q}")
        Q -= M1 @ J1.t() @ dL
        iters +=1
    #print(f"Updating v1_col, Jacobian {J(gs,torch.squeeze(x_col))}")
    #print(f"Updating v1_col, Jacobian^T {J(gs,torch.squeeze(x_col)).t()}")

    # half step for velocity
    Q_col = Q.reshape(-1,1)
    v1_half = (Q_col - x_col)/h
    x_col = Q_col
    J1 = J(gs, torch.squeeze(x_col))

    # getting the level
    J2 = J(gs, torch.squeeze(Q))
    P = J1 @ M1 @ J1.t()
    T = J1 @ (2/h * v1_half - M1 @ DV_col)

    #solving the linear system
    L = torch.linalg.solve(P,T)

    v1_col = v1_half - h/2 * DV_col - h/2 * J2.t()@L


    # print(f"Error = {G1(x_col + h*( v1_col + h/2 * torch.inverse(M) @ J1.reshape(-1,1) @ lam))}")
    # # updating v
    # print(f"lam = {lam}")
    # print(f"Updating v1_col, Jacobian^T {J(gs,torch.squeeze(x_col)).t}")

    return torch.squeeze(x_col), torch.squeeze(v1_col)

In [ ]:
@torch.compile(mode = "default")
def gBAOAB_step_exact(q_init,p_init,F, GO, h,M, gamma, k, kr):

    errors = []
    # setting up variables
    M1 = torch.inverse(M)
    R = torch.randn(len(q_init))
    p = p_init
    q = q_init
    a2 = torch.exp(torch.tensor(-gamma*h))
    b2 = torch.sqrt(k*(1-a2**(2)))

    # doing the initial p-update
    J1 = jacobian(GO,torch.squeeze(q))
    G = J1
    L1 = torch.eye(len(q_init)) - torch.transpose(G,0,1) @ torch.inverse(G@ M1@ torch.transpose(G,0,1)) @ G @ M1
    p -=  h/2 * L1 @ F(q)


    # doing the first RATTLE step
    for i in range(kr):
      q, p= rattle_step(q, p, h/(2*kr), M, gs, 10**(-15))
    errors.append(torch.abs(GO(torch.squeeze(q))).mean())

    # the second p-update - (O-step in BAOAB)
    J2 = jacobian(GO,torch.squeeze(q))
    G = J2
    L2 = torch.eye(len(q_init)) - torch.transpose(G,0,1) @ torch.inverse(G@ M1@ torch.transpose(G,0,1)) @ G @ M1
    p = a2* p + b2* M**(1/2) @L2 @ M**(1/2) @ R

    # doing the second RATTLE step
    for i in range(kr):
      q, p = rattle_step(q, p, h/(2*kr), M, gs, 10**(-15))
    errors.append(torch.abs(GO(torch.squeeze(q))).mean())
    # the final p update
    J3= jacobian(GO,torch.squeeze(q))
    G = J3
    L3 = torch.eye(len(q_init)) - torch.transpose(G,0,1) @ torch.inverse(G@ M1@ torch.transpose(G,0,1)) @ G @ M1
    p -=  h/2 * L3 @ F(q)

    return q,p, np.array(errors).mean()

In [ ]:
@torch.compile(mode = "default")
def gBAOAB_integrator(q_init,p_init,F, gs, h,M, gamma, k, steps,kr):
    positions = []
    velocities = []
    errors = []
    q = q_init
    p = p_init
    for i in range(steps):
        print(f"STEP {i}")
        q, p, error = gBAOAB_step_exact(q,p, F,gs, h,M, gamma, k,kr)
        positions.append(q)
        velocities.append(p)
        errors.append(error)

    return positions, velocities, errors

In [ ]:
@torch.compile(mode = "default")
def multi_gBAOAB(q_inits, p_inits,F, gs, h,M, gamma, k, step,kr):
  ps = []
  vs = []
  for i in range(len(q_inits)):
    positions, velocities = gBAOAB_integrator(q_inits[i],p_inits[i],F, gs, h,M, gamma, k, step,kr)
    ps.append(positions)
    vs.append(velocities)
  return ps, vs

In [ ]:
@torch.compile(mode = "default")
def potential_1(x):
    # setting potential to (x^2 -1) + ...
    y = torch.squeeze(x**2)
    return torch.dot(y-1,y-1)

@torch.compile(mode = "default")
def force_1(x):
    return torch.squeeze(jacobian(potential_1, x), dim = 0)


# Setting random points and running

In [ ]:
p_init= torch.tensor([0.,0.,0.])
q_init = torch.sqrt(torch.tensor([0.5,0.5,0]))


gs = [lambda x: x[0] ** 2 + x[1]**2 + x[2]** 2 - 1 ]
M = torch.eye(3)

In [ ]:
gamma_phis = []
gamma_thetas = []
for gamma in [0.0001,0.001,0.1,1,10,100]:
  positions, _, _ = gBAOAB_integrator(q_init,p_init,force_1, G(gs), 0.01,M, gamma, 1, 1000000,1)
  phi_s = [phi(p) for p in positions]
  theta_s = [theta(p) for p in positions]
  phis_cum = np.cumsum(phi_s)/range(1,len(phi_s)+1)
  thetas_cum = np.cumsum(theta_s)/range(1,len(theta_s)+1)
  gamma_thetas.append(thetas_cum[-1])
  gamma_phis.append(phis_cum)

In [ ]:
beta_phis = []
beta_thetas = []
for beta in [0.0001,0.001,0.1,1,10,100]:
  positions, _, _ = gBAOAB_integrator(q_init,p_init,force_1, G(gs), 0.01,M, 1, beta, 1000000,1)
  phi_s = [phi(p) for p in positions]
  theta_s = [theta(p) for p in positions]
  phis_cum = np.cumsum(phi_s)/range(1,len(phi_s)+1)
  thetas_cum = np.cumsum(theta_s)/range(1,len(theta_s)+1)
  beta_thetas.append(thetas_cum[-1])
  beta_phis.append(phis_cum)